In [1]:
import numpy as np
import random
import string
import math
from collections import defaultdict
import json

In [2]:
def gen_word(word_length):
    word_len = np.random.randint(*word_length)
    return ''.join(random.sample(string.ascii_lowercase, word_len))

In [3]:
topic_size = 6
num_topics = 20

topics = [
    [gen_word((2,6)) for _ in range(topic_size)] for _ in range(num_topics)
]

In [4]:
topics

[['ph', 'abfz', 'ez', 'xu', 'dtqw', 'md'],
 ['xlju', 'gdj', 'mxs', 'djugv', 'lpyxh', 'kwgdt'],
 ['bzljx', 'sz', 'ghm', 'fuzd', 'wx', 'zcn'],
 ['zm', 'fyhzr', 'jmftg', 'bsp', 'dzc', 'ckpe'],
 ['bva', 'cvstd', 'oqpid', 'vmwtk', 'sgnu', 'dvjz'],
 ['gbj', 'fdck', 'pau', 'srhf', 'au', 'ux'],
 ['nbdat', 'zn', 'zisa', 'fmks', 'qhclf', 'kubym'],
 ['cera', 'cquw', 'rm', 'lskc', 'jo', 'uqw'],
 ['oi', 'gwvpo', 'ztxw', 'hcfiu', 'wtmp', 'vfoip'],
 ['kfnu', 'jy', 'qhwvx', 'hl', 'roydt', 'qsup'],
 ['bxuwj', 'ib', 'afcm', 'xprun', 'rp', 'rnef'],
 ['vbdp', 'hn', 'yo', 'pyeqg', 'jwfum', 'jbqil'],
 ['fxp', 'dji', 'yfgbp', 'tpqeg', 'rljxg', 'jg'],
 ['gl', 'jmxr', 'mg', 'fxnj', 'wmau', 'gilwv'],
 ['xda', 'ebpcv', 'qu', 'gaebv', 'spze', 'dalku'],
 ['ev', 'oyw', 'elx', 'uznv', 'vpr', 'vtx'],
 ['lrt', 'irq', 'dq', 'roq', 'ms', 'zjh'],
 ['ynx', 'ixh', 'mdsw', 'zf', 'wzop', 'mztlk'],
 ['kdp', 'nw', 'xa', 'nd', 'askc', 'cxoyw'],
 ['yfem', 'tplnd', 'qzl', 'ytm', 'jsd', 'no']]

In [23]:
def generate_sentence(m_topics):
    sent = []
    prefix_topics = 1
    suffix_topics = 1
    
    for i in range(prefix_topics):
        sent += random.sample(topics[random.choice(range(num_topics))], random.randint(0,2))
    
    mention = random.choice(random.choice(topics))
    
    sent += random.sample(topics[random.choice(m_topics)], random.randint(0,2)) + ["\t", mention, "\t"] + random.sample(topics[random.choice(m_topics)], random.randint(0,2))
    
    for i in range(suffix_topics):
        sent += random.sample(topics[random.choice(range(num_topics))], random.randint(0,2))
      
    return sent
    

In [24]:
m_topics = random.choices(range(num_topics), k=2)
m_topics, generate_sentence(m_topics)

([2, 19], ['sz', '\t', 'vtx', '\t', 'jsd', 'fdck'])

In [25]:
def generate_to_file(file_path, count=1000):
    with open(file_path, 'w') as out:
        for i in range(count):
            m_topics = random.choices(range(num_topics), k=2)
            topics_str = "_".join(map(str, m_topics))
            for j in range(random.randint(2, 4)):
                sent = generate_sentence(m_topics)
                out.write(topics_str + "\t" + ' '.join(sent) + '\n')

In [26]:
generate_to_file('../data/fake_data_train.tsv')
generate_to_file('../data/fake_data_valid.tsv', count=500)          
generate_to_file('../data/fake_data_test.tsv', count=500) 

In [27]:
with open('../data/fake_ft_data.tsv', 'w') as out:
    for i in range(1000):
        m_topics = random.choices(range(num_topics), k=2)
        topics_str = "_".join(map(str, m_topics))
        for j in range(random.randint(2, 4)):
            sent = generate_sentence(m_topics)
            out.write(' '.join(sent) + '\n')

In [28]:
#!pip install git+https://github.com/facebookresearch/fastText.git

In [29]:
import fastText

In [30]:
model = fastText.train_unsupervised(
    input='../data/fake_ft_data.tsv', minCount=0, bucket=1000, dim=30
)

In [31]:
len(model.get_words())

121

In [32]:
model.save_model("../data/fake_ft_model.bin")

In [33]:
from gensim.models import FastText
from gensim.models.utils_any2vec import ft_ngram_hashes
from gensim.models.utils_any2vec import compute_ngrams

In [34]:
ft = FastText.load_fasttext_format("../data/fake_ft_model.bin")

/home/generall/sources/Sci/ml/allen_evn/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  """Entry point for launching an IPython kernel.


In [35]:
ft.save('../data/gensim_fake_ft.model')

In [36]:
ft2 = FastText.load('../data/gensim_fake_ft.model')

In [37]:
ft2.num_ngram_vectors

/home/generall/sources/Sci/ml/allen_evn/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `num_ngram_vectors` (Attribute will be removed in 4.0.0, use self.wv.num_ngram_vectors instead).
  """Entry point for launching an IPython kernel.


1000